In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random


In [3]:
dfFlow = pd.read_csv("./data/food_flow_network_US-cities.csv")
dfRegions = pd.read_csv("./data/regions_description.csv")
dfDiversity = pd.read_csv("./data/diversity_US-cities.csv")

print(dfDiversity.head())

    ID  sector  diversity_2012  diversity_2013  diversity_2014  diversity_2015
0  101  animal        0.504571        0.505614        0.505372        0.505256
1  101    crop        0.724984        0.726144        0.768855        0.748384
2  101    feed        0.570448        0.565756        0.568316        0.565534
3  101    meat        0.734430        0.724947        0.739431        0.735322
4  102  animal        0.378877        0.379139        0.381346        0.381900


In [4]:
print("Food Flows")
print(dfFlow.head())
print("Region Description")
print(dfRegions.head())
print("Region Diversity")
print(dfDiversity.head())

Food Flows
   origin  destination  sector  tons_2012  tons_2013  tons_2014  tons_2015
0     101          101  animal   5.047236   5.020935   5.099104   5.206390
1     101          101    crop   0.054580   0.053380   0.059174   0.059911
2     101          101    feed   6.001596   6.192498   6.237931   6.369929
3     101          101    meat   0.453039   0.458094   0.460049   0.461909
4     101          102  animal  39.448790  41.622335  45.323130  47.220295
Region Description
    ID  State_FIPS                                    Name Region_type
0  101           1        Anniston-Oxford-Jacksonville, AL         MSA
1  102           1         Birmingham-Hoover-Talladega, AL         CSA
2  103           1  Chattanooga-Cleveland-Dalton, TN-GA-AL         CSA
3  104           1          Columbus-Auburn-Opelika, GA-AL         CSA
4  105           1             Dothan-Enterprise-Ozark, AL         CSA
Region Diversity
    ID  sector  diversity_2012  diversity_2013  diversity_2014  diversity_201

In [5]:
from collections import defaultdict 
nodeDiversity = defaultdict(list)

for index, row in dfDiversity.iterrows():
    nodeDiversity[row['ID']].append(row['diversity_2015'])

for id in nodeDiversity:
    nodeDiversity[id] = sum(nodeDiversity[id])/len(nodeDiversity[id])

nodeDiversity

defaultdict(list,
            {101: 0.6386241023734112,
             102: 0.549209704958835,
             103: 0.7438011247487074,
             104: 0.7464955525451153,
             105: 0.6933824045803116,
             106: 0.8004581650637347,
             107: 0.6371571915152963,
             108: 0.7362627195728422,
             109: 0.4415992899581836,
             110: 0.7321292050949304,
             111: 0.5818696572209274,
             199: 0.6904993052952562,
             401: 0.4879679205391887,
             402: 0.3939155018966649,
             403: 0.6364410730412061,
             404: 0.3746487605263097,
             405: 0.40361330484942387,
             406: 0.3463563640222104,
             407: 0.4088496792018141,
             499: 0.4727137617474785,
             501: 0.6880888975536087,
             502: 0.6943334014398804,
             503: 0.6044743621948205,
             504: 0.59242354509865,
             505: 0.6181270138847145,
             506: 0.71725222504412

In [6]:
adjacency_dict = dict()

for index, row in dfFlow.iterrows():
    if row['origin'] not in adjacency_dict:
        adjacency_dict[row['origin']] =[]
    curTuple = (row['destination'], row['tons_2015'])
    adjacency_dict[row['origin']].append(curTuple)


adjacency_dict

{101: [(101, 5.20638975),
  (101, 0.0599110025433148),
  (101, 6.3699289599999975),
  (101, 0.4619085487258096),
  (102, 47.22029499999999),
  (102, 0.8326253492962324),
  (102, 31.981184),
  (102, 0.3249081081081081),
  (103, 0.2403120840336134),
  (103, 0.6335806746971966),
  (103, 1.5263013333333335),
  (103, 0.3392093049128949),
  (104, 1.7439564264705882),
  (104, 0.0529268884321452),
  (104, 1.0054566753232723),
  (105, 15.61916925),
  (105, 0.4883646201288024),
  (105, 6.3699289599999975),
  (105, 0.9817133032509748),
  (106, 1.213341943251666),
  (106, 19.10978688),
  (106, 0.5800536956633142),
  (107, 1.73546325),
  (107, 0.0596672607007425),
  (107, 12.73985792),
  (107, 0.4107040424327712),
  (108, 34.709265),
  (108, 3.454669190536935),
  (108, 50.95943168),
  (108, 2.672872863938475),
  (109, 0.09669),
  (110, 3.4709265),
  (110, 0.188741557659226),
  (110, 25.47971584),
  (110, 1.4773010416004515),
  (111, 13.883706),
  (111, 0.2403893405588709),
  (111, 6.369928959999997

In [7]:

def calculate_total_incoming_weight(adjacency_dict):
    total_weight = {node: 0 for node in adjacency_dict}
    for node, connections in adjacency_dict.items():
        for connected_node, weight in connections:
            total_weight[connected_node] += weight

    return total_weight

def simulate_normal_distribution_removals(adjacency_dict, mean_percentage, std_dev_percentage):
    modified_adj_dict = {node: connections.copy() for node, connections in adjacency_dict.items()}
    initial_weights = calculate_total_incoming_weight(modified_adj_dict)
    all_connections = [(node, conn) for node, connections in modified_adj_dict.items() for conn in connections]
    num_connections = len(all_connections)
    num_removals = int(np.random.normal(mean_percentage * num_connections, std_dev_percentage * num_connections))
    num_removals = max(0, min(num_removals, num_connections))
    for _ in range(num_removals):
        if not all_connections:
            break
        removed_connection = random.choice(all_connections)
        modified_adj_dict[removed_connection[0]].remove(removed_connection[1])
        all_connections.remove(removed_connection)
    final_weights = calculate_total_incoming_weight(modified_adj_dict)
    resilience = {node: final_weights[node] / initial_weights[node] for node in modified_adj_dict}

    return resilience
mean_percentage = 0.5 
std_dev_percentage = 0.1
normal_distribution_resilience = simulate_normal_distribution_removals(adjacency_dict, mean_percentage, std_dev_percentage)
normal_distribution_resilience



{101: 0.43442953115024674,
 102: 0.42937653332306425,
 103: 0.37672023538693017,
 104: 0.4522549012251319,
 105: 0.35548419103959406,
 106: 0.5119504439645549,
 107: 0.3167495332282508,
 108: 0.4491602594613707,
 109: 0.5031449590122677,
 110: 0.279093820175094,
 111: 0.4144935845712878,
 199: 0.4821598345315864,
 401: 0.6935864516921061,
 402: 0.4946200005089084,
 403: 0.3025947486768956,
 404: 0.20463172021980572,
 405: 0.376596462088839,
 406: 0.6363397153728295,
 407: 0.7098687009996854,
 499: 0.4135855713054444,
 501: 0.25420127429444817,
 502: 0.3022745518416552,
 503: 0.6168880964363862,
 504: 0.28395660402106354,
 505: 0.6795289009223983,
 506: 0.6661162358790115,
 507: 0.6228611762730615,
 599: 0.3159311631460983,
 601: 0.19391617409828063,
 602: 0.3983293428523673,
 603: 0.7917723996827313,
 604: 0.21139208912831298,
 605: 0.5227081485903571,
 606: 0.2210398595856553,
 607: 0.45685899200611796,
 608: 0.44747088063807794,
 609: 0.31415568824186024,
 610: 0.5276264667046427,
 6